In [25]:
import numpy as np
import pandas as pd
import math
import random
import pickle
import json

# to make the experimens replicable
random.seed(123456)

In [26]:
df_uffici = pd.read_csv('../../data/dg_struttura.csv', dtype=str)
id_label_dict = dict(zip(df_uffici['ID'].values, df_uffici['NOME'].values))

In [29]:
df = pd.read_pickle('../../data/atti.pkl')

In [30]:
df.head()

,CODICE_PRATICA,DATA_ATTO,DESCRIZIONE,DESCRIZIONE_TIPO_ATTO,DESCRIZIONE_TIPO_PRATICA,ID_TIPO_PRATICA,LINK_CONTENUTO_BINARIO,OGGETTO,PERSONA,STATUS,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,publisher
0,20000000011999AD00000005121,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. ALESSANDRO ANNUNZIATI (MATR. 14117). ASPE...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
1,20000000021999AD00000005123,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. GIOVANNI ORLANDINI (MATR. 14311). ASPETTA...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
2,20000000031999AD00000005018,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. LEONELLO TOCCAFONDI. CONGEDO STRAORDINARI...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
3,20000000041999AD00000005116,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIPENDENTI REGIONALI DIVERSI. CONGEDO STRAORDI...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
4,20000000051999AD00000005127,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. SILVIA LORENZI (MATR. 13366). CONGEDO STR...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows


In [31]:
df.shape

(184381, 14)

In [32]:
# df = df[df['DATA_ATTO'].dt.year > 2007]
# df.shape

### Documents per Office

In [33]:
grouped = df.groupby(['UFFICIO_DG']).count()['CODICE_PRATICA'].sort_values(ascending=False)
grouped

UFFICIO_DG
50073    15753
50117    14061
50000    12443
50004    11513
50003    11148
50005     9869
01937     8799
50079     7981
50118     7258
50006     6790
50001     6231
50119     5999
01934     5885
01943     5189
50125     4893
50116     4299
01928     3889
01946     3857
50007     3597
01025     3257
50002     2894
50083     2867
50102     2299
50115     2179
50123     2160
50069     1985
01931     1748
50202     1603
02090     1568
01027     1528
50127     1445
50121     1303
01923     1142
50112     1102
99999     1060
50124      972
50201      902
50113      877
50114      678
50122      603
50044      464
50200      186
50026      105
Name: CODICE_PRATICA, dtype: int64

we remove the observations for the following offices

In [52]:
offices = grouped.index[-5:]
offices

Index(['50114', '50122', '50044', '50200', '50026'], dtype='object', name='UFFICIO_DG')

In [53]:
for o in offices:
    df = df[df['UFFICIO_DG'] != o]

In [54]:
df.shape

(179594, 14)

## Dataset Creation

Indipendent Variable:
- 'OGGETTO'
- 'UFFICIO_DG'
- 'DESCRIZIONE_TIPO_ATTO'
- 'DESCRIZIONE_TIPO_PRATICA'
- 'publisher'

In [55]:
df_dataset = df[['OGGETTO', 'UFFICIO_DG']]

In [56]:
df_dataset.shape

(179594, 2)

In [57]:
samples = np.array([x[0] for x in df_dataset.as_matrix()])
labels = np.array([x[1] for x in df_dataset.as_matrix()])

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [58]:
print(samples[10])
print(labels[10])

ELENCO REGIONALE DELLE AZIENDE PRODUTTRICI E FORNITRICI DI PROTESI ED AUSILI DI CUI AL DECRETO N. 1317 DEL 20.03.1998: MODIFICHE ED INTEGRAZIONI
01943


In [59]:
labels = np.array([id_label_dict[x] if x in id_label_dict else 'office_{}'.format(x) for x in labels])

In [60]:
indexes = np.array(range(len(samples)))
np.random.shuffle(indexes)

In [61]:
samples = samples[indexes]
labels = labels[indexes]

### Transform labels to ids

In [62]:
ids_labels = dict(enumerate(sorted(list(set(labels)))))
labels_ids = {v:k for k,v in ids_labels.items()}

In [63]:
labels_ids

{'ALTRI UFFICI': 0,
 'AVVOCATURA REGIONALE                                  ': 1,
 'D.G.  AVVOCATURA                                      ': 2,
 "D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SVILUPPO D": 3,
 'D.G. DIRITTI DI CITTADINANZA E COESIONE SOCIALE       ': 4,
 'D.G. ORGANIZZAZIONE                                   ': 5,
 'D.G. ORGANIZZAZIONE E RISORSE                         ': 6,
 'D.G. POLITICHE TERRITORIALI, AMBIENTALI E PER LA MOBIL': 7,
 'D.G. PRESIDENZA                                       ': 8,
 'DIPARTIMENTO BILANCIO E FINANZE                       ': 9,
 'DIPARTIMENTO ORGANIZZAZIONE                           ': 10,
 'DIPARTIMENTO ORGANIZZAZIONE E RISORSE                 ': 11,
 'DIPARTIMENTO POLITICHE FORMATIVE E BENI CULTURALI     ': 12,
 'DIPARTIMENTO POLITICHE TERRITORIALI E AMBIENTALI      ': 13,
 'DIPARTIMENTO PRESIDENZA AFFARI LEGISLATIVI E GIURIDICI': 14,
 'DIPARTIMENTO SALUTE E POLITICHE SOLIDARIETA           ': 15,
 'DIPARTIMENTO SVILUPPO ECONOMICO           

In [64]:
generale = []
others = []
for k,_ in labels_ids.items():
        if 'D.G' in k:
            generale.append(k)
        elif 'GENERALE' in k:
            generale.append(k)
        else:
            others.append(k)
        

In [65]:
generale

['D.G.  AVVOCATURA                                      ',
 "D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SVILUPPO D",
 'D.G. DIRITTI DI CITTADINANZA E COESIONE SOCIALE       ',
 'D.G. ORGANIZZAZIONE                                   ',
 'D.G. ORGANIZZAZIONE E RISORSE                         ',
 'D.G. POLITICHE TERRITORIALI, AMBIENTALI E PER LA MOBIL',
 'D.G. PRESIDENZA                                       ',
 'DIREZIONE GENERALE BILANCIO E FINANZE                 ',
 'DIREZIONE GENERALE DIREZIONE GENERALE DELLA GIUNTA REGIONALE',
 'DIREZIONE GENERALE DIRITTO ALLA SALUTE E POLITICHE DI ',
 'DIREZIONE GENERALE ORGANIZZAZIONE E SISTEMA INFORMATIV',
 'DIREZIONE GENERALE POLITICHE FORMATIVE, BENI E ATTIVIT',
 'DIREZIONE GENERALE POLITICHE TERRITORIALI E AMBIENTALI',
 'DIREZIONE GENERALE SVILUPPO ECONOMICO                 ']

In [66]:
others

['ALTRI UFFICI',
 'AVVOCATURA REGIONALE                                  ',
 'DIPARTIMENTO BILANCIO E FINANZE                       ',
 'DIPARTIMENTO ORGANIZZAZIONE                           ',
 'DIPARTIMENTO ORGANIZZAZIONE E RISORSE                 ',
 'DIPARTIMENTO POLITICHE FORMATIVE E BENI CULTURALI     ',
 'DIPARTIMENTO POLITICHE TERRITORIALI E AMBIENTALI      ',
 'DIPARTIMENTO PRESIDENZA AFFARI LEGISLATIVI E GIURIDICI',
 'DIPARTIMENTO SALUTE E POLITICHE SOLIDARIETA           ',
 'DIPARTIMENTO SVILUPPO ECONOMICO                       ',
 'DIREZIONE AGRICOLTURA E SVILUPPO RURALE',
 'DIREZIONE AMBIENTE ED ENERGIA',
 "DIREZIONE ATTIVITA' PRODUTTIVE",
 'DIREZIONE DIFESA DEL SUOLO E PROTEZIONE CIVILE',
 'DIREZIONE DIRITTI DI CITTADINANZA E COESIONE SOCIALE',
 'DIREZIONE ISTRUZIONE E FORMAZIONE',
 'DIREZIONE LAVORO',
 'DIREZIONE ORGANIZZAZIONE E SISTEMI INFORMATIVI',
 "DIREZIONE POLITICHE MOBILITA', INFRASTRUTTURE E TRASPORTO PUBBLICO LOCALE",
 'POLITICHE AMBIENTALI, ENERGIA E CAMBIAMEN

In [67]:
with open('../../data/dataset/label_index.json', 'w') as f:
    json.dump(labels_ids,f)

In [68]:
encoded_labels = np.array([labels_ids[x] for x in labels])

### Transform the samples into ids

Removed numbers

In [69]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')

punctuation = ['-', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}', '’', '”', '“', '``', "''"]
stop_words = set(stopwords.words('italian'))
stop_words.update(punctuation)

[nltk_data] Downloading package stopwords to /home/fabio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fabio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [70]:
def hasnumbers(value):
    return any(c.isdigit() for c in value)

In [71]:
def tokenize_sample(samples, remove_stopwords=True, tokenizer=word_tokenize):
    for sample in samples:
        words = []
        sample = sample.replace('`', ' ')
        sample = sample.replace("'", " ")
        sample = sample.replace("”", ' ')
        sample = sample.replace("“", ' ')
        for w in tokenizer(sample):
#             if not hasnumbers(w) and len(w) > 2:
            w = w.replace('_', '')
            if remove_stopwords:
                if w not in stop_words:
                    words.append(w.lower())
            else:
                words.append(w.lower())
        yield words

In [72]:
tokenizer = RegexpTokenizer(r'\w+')

In [73]:
samples_tokenized = list(tokenize_sample(samples, remove_stopwords=False))

In [74]:
from collections import Counter

counter = Counter()

for sentence in samples_tokenized:
    counter.update(sentence)

In [75]:
counter.most_common()[:30]

[('.', 400788),
 ('di', 293060),
 ('-', 134639),
 ('e', 133284),
 ('del', 120795),
 ('per', 103675),
 (',', 94597),
 ('della', 67681),
 ('n.', 63868),
 (')', 55068),
 ('``', 51012),
 ('(', 50303),
 ('in', 49295),
 ("''", 48857),
 ('a', 48186),
 ('dell', 42154),
 ('la', 39951),
 ('regionale', 36958),
 ('al', 35256),
 ('l.r', 31063),
 ('delle', 30772),
 ('dei', 28948),
 ('comune', 28942),
 ('impegno', 28152),
 ('alla', 27127),
 ('approvazione', 26935),
 ('art', 26778),
 (':', 26490),
 ('l', 26035),
 ('toscana', 25893)]

The words are enumerated in ascending order so that we can select the top k words during the classification task

In [76]:
counter.__len__()

136279

In [77]:
words = [w for w,c in counter.most_common() if c > 2]

In [78]:
words[:30]

['.',
 'di',
 '-',
 'e',
 'del',
 'per',
 ',',
 'della',
 'n.',
 ')',
 '``',
 '(',
 'in',
 "''",
 'a',
 'dell',
 'la',
 'regionale',
 'al',
 'l.r',
 'delle',
 'dei',
 'comune',
 'impegno',
 'alla',
 'approvazione',
 'art',
 ':',
 'l',
 'toscana']

In [79]:
len(words) + 3

37658

In [80]:
id_word_dict = dict(enumerate([w for w in words],3))
word_id_dict = {v:k for k,v in id_word_dict.items()}

In [81]:
with open('../../data/dataset/id_word_dict.json','w') as f:
    json.dump(id_word_dict,f)

In [82]:
pad_char = 0
start_char=1
oov_char=2

In [83]:
def to_sequece_of_ids(tokenized_samples, word_id_dict):
    results = []
    for sample in tokenized_samples:
        encoded_sample = []
        for w in sample:
            if w in word_id_dict:
                encoded_sample.append(word_id_dict[w])
            else:
                encoded_sample.append(oov_char)
        results.append(encoded_sample)
    return results

In [84]:
encoded_samples = np.array(to_sequece_of_ids(samples_tokenized, word_id_dict))

In [85]:
encoded_samples[:4]

array([list([273, 21, 379, 390, 124, 116, 21673, 5, 74, 20, 8, 19, 762, 4, 86, 8, 31, 900, 14, 2, 12, 3, 135, 15, 202, 144, 162, 20, 3]),
       list([22, 3, 7373, 3, 26, 6, 34, 125, 682, 1185, 211]),
       list([60, 4, 3863, 105, 1717, 171, 35, 5696, 6, 467, 24, 1377, 1524, 4, 61, 21, 48, 11, 29607, 7, 19443, 3, 413, 7, 730, 879, 6, 28, 7, 171, 141, 4, 176, 3]),
       list([95, 464, 2, 8, 31, 395, 4711, 13, 230, 13389, 24, 2774, 4, 367, 16])],
      dtype=object)

In [86]:
max(np.max(encoded_samples))

37655

## Create Train and Test Set

- shuffle data
- split the dataset into 80-20

In [87]:
from sklearn.model_selection import StratifiedShuffleSplit

In [88]:
split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_test.split(encoded_samples, encoded_labels):
    train_samples, test_samples = encoded_samples[train], encoded_samples[test]
    train_labels, test_labels = encoded_labels[train], encoded_labels[test]

## Save the Dataset

In [89]:
np.savez_compressed('../../data/dataset/atti_dataset.npz',
                   x_train=train_samples, y_train=train_labels,
                   x_test=test_samples, y_test=test_labels
                   )